In [ ]:
import kutils
from kutils import model_helper as mh
from kutils import applications as apps
from kutils import image_utils as iu
from kutils import tensor_ops as ops
import pandas as pd

### KonCept512 model
3 fully-connected layers + dropouts on top of the Global Average Pooled features from InceptionResNet-v2

In [ ]:
aux_root = '/mnt/home/research/data/'
data_root = '/mnt/home/research/koniq/'
ids = pd.read_csv(data_root + 'metadata/koniq10k_distributions_sets.csv')

from keras.models import Model

# build scoring model
base_model, preprocess_fn = apps.get_model_imagenet(apps.InceptionResNetV2)
head = apps.fc_layers(base_model.output, name='fc', 
                      fc_sizes      = [2048, 1024, 256, 1], 
                      dropout_rates = [0.25, 0.25, 0.5, 0])    

model = Model(inputs = base_model.input, outputs = head)
print('[done]')

In [ ]:
# parameter of the generator
pre = lambda im: preprocess_fn(iu.ImageAugmenter(im, remap=False).fliplr().result)
gen_params = dict(batch_size  = 32, 
                  data_path   = data_root+'images/512x384/',
                  process_fn  = pre, 
                  input_shape = (384,512,3),
                  outputs     = ('MOS',))

# Wrapper for the model, helps with training and testing
helper = mh.ModelHelper(model, 'KonCept512', ids, 
                     loss='MSE', metrics=["MAE", ops.plcc_tf],
                     monitor_metric = 'val_loss', 
                     monitor_mode   = 'min', 
                     multiproc   = True, workers = 5,
                     logs_root   = aux_root + 'logs/koniq',
                     models_root = aux_root + 'models/koniq',
                     gen_params  = gen_params)

print '[done]'

#### Train model

In [ ]:
helper.train(lr=1e-4, epochs=100)

161/221 [====================>.........] - ETA: 1:17 - loss: 146.3941 - mean_absolute_error: 9.5622 - plcc_tf: 0.7169

#### Test

In [ ]:
test_rating_model(helper);